### 1. Download dataframe for Toronto boroughs with coordinates from week 1-3

In [ ]:
#!conda install -c conda-forge geopy --yes

In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [20]:
# Data wrangling modules
import io
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Import plotting libraries

In [21]:
import json # For handling JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # Map rendering library 

Fetch **Borough** data from dataframe from Github

In [22]:
url = 'https://raw.githubusercontent.com/wgova/Coursera_Capstone/master/toronto_df.csv'
toronto_df = pd.read_csv(url)
toronto_df.head(3)

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:
print('Uniques Boroughs: {} .'.format(len(toronto_df['Borough'].unique())))
print('List of different categories:')
list(toronto_df['Borough'].unique())

Plot geotagged map of Toronto with Toronto Boroughs markers 

In [7]:
# create empty map of Toronto using latitude and longitude values
lat = 43.7615390 ; lon = -79.411079
Toronto_boroughs_map = folium.Map(location=[lat, lon], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], 
                           toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_boroughs_map)  
    
Toronto_boroughs_map

In [8]:
#Central Toronto

# selecting only neighborhoods regarding to "Scarborough" borough.
toronto = toronto_df[toronto_df['Borough'] == 'Central Toronto']
#c_toronto = toronto_df[toronto_df['Borough'] == 'Downtown Toronto']
#toronto = toronto_df[toronto_df['Borough'] == 'North York','East York','East Toronto','Central Toronto','Downtown Toronto','York','West Toronto',"Queen's Park",'Mississauga','Etobicoke']
toronto = toronto_df.reset_index(drop=True).drop(columns = 'Unnamed: 0')
#toronto = toronto_df[toronto_df['Borough'] == 'North York']
toronto.head(3)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711


### 2. Make an API call to Foursquare to get information on venues in Central Toronto

In [16]:
Client_ID = 'V1IHZGU2XZ2XAEHV0G3EZIVIQTFFLCMPWNI3I3LRUL3PYRAA'
Client_Secret = 'OSMB212BOD4NKSCN4IYRRIAFOAFVTEO0KM0EJ1245PK0O0QJ'
Version = '20181026'

Define a function to fetch coordinates from FOURSQUARE

In [17]:
def api_call_4sqr (postal_code_list, neighbourhood_list, lat_list, lng_list, LIMIT = 50000, radius = 10000):
    api = []
    counter = 0
    for postal_code, neighbourhood, lat, lng in zip(postal_code_list, neighbourhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_ID, Client_Secret, Version, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        api_dict = {}
        api_dict['Postalcode'] = postal_code; api_dict['Neighbourhood(s)'] = neighbourhood; 
        api_dict['Latitude'] = lat; api_dict['Longitude'] = lng;
        api_dict['API_calls'] = results;
        api.append(api_dict)
        counter += 1
    return api;

In [19]:
Toronto_4sqr = api_call_4sqr(list(toronto['Postalcode']),list(toronto['Neighbourhood']),
                             list(toronto['Latitude']),list(toronto['Longitude']),)

SSLError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=V1IHZGU2XZ2XAEHV0G3EZIVIQTFFLCMPWNI3I3LRUL3PYRAA&client_secret=OSMB212BOD4NKSCN4IYRRIAFOAFVTEO0KM0EJ1245PK0O0QJ&v=20181026&ll=43.8066863,-79.19435340000003&radius=10000&limit=50000 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')",),))

In [ ]:
#Use pickle to pickle results and store them in local drive
import pickle
with open("Toronto_4sqr.txt", "wb") as fp:   
    pickle.dump(Toronto_4sqr, fp)   
# Unpickle results
with open("Toronto_4sqr.txt", "rb") as fp:   
    Toronto_4sqr = pickle.load(fp)

print("Foursquare API call results saved in: ")
%pwd

Plot the map of Central Toronto superimposed onto the one for Toronto

In [ ]:
from folium import IFrame
lat = 43.72021610 ; lon = -79.5395769
map_Toronto_Central = folium.Map(location=[lat, lon], zoom_start=10.5)

# Add a popup label for the map
text = 'Boroughs in Central Toronto'
iframe = folium.IFrame(text, width=700, height=50)
popup = folium.Popup(iframe, max_width=1000)
Text = folium.Marker(location=[lat,lon], popup=popup,
                     icon=folium.Icon(icon_color='green'))

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng], radius = 6, popup = label, color ='red', fill = True,fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(Toronto_boroughs_map)

#Toronto_boroughs_map.add_child(Text)
#Toronto_boroughs_map.save('dToronto_Central.html')
#Toronto_boroughs_map


### Data cleaning to convert the raw Foursquare data into dataframes 

In [ ]:
# Function to create dataframe
def venues_results(Toronto_4sqr):
    venues_df = pd.DataFrame(columns = ['Postalcode', 'Neighbourhood','Latitude', 'Longitude','Venue', 
                                        'Tips for venue', 'Venue Category', 'Distance'])
    
    for hood_dict in Toronto_4sqr: # Store neighborhood names in a dictionary
        postal_code = hood_dict['Postalcode']; 
        hood = hood_dict['Neighbourhood(s)'];
        lat = hood_dict['Latitude']; 
        lon = hood_dict['Longitude']
                
        for venue_dict in hood_dict['API_calls']: #Store venue names in a dictionary
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            distance = venue_dict['venue']['location']['distance']
            category =  venue_dict['venue']['categories'][0]['name']
            
            venues_df = venues_df.append({
                'Postalcode': postal_code, 'Neighbourhood': hood,'Latitude': lat, 'Longitude':lon,
                'Venue': name, 'Tips for venue': summary,'Venue Category': category, 'Distance': distance
            },ignore_index = True)
    return(venues_df)

Save a copy of the dataframe results to a csv file

In [ ]:
venues_toronto = venues_results(Toronto_4sqr)

In [ ]:
venues = venues_toronto['Venue'].unique()
venues = len(venues) ; print('Venues:',venues)
hoods = venues_toronto['Neighbourhood'].unique()
hoods = len(hoods) ; print('Neighbourhoods:', hoods)


In [ ]:
venues_toronto.to_csv('Tips - Central Toronto.csv')
venues_toronto.tail()

In [ ]:
venues_toronto = pd.read_csv('Tips - Central Toronto.csv')
#venues_toronto.head()
list(venues_toronto['Venue Category'].unique())

In [ ]:
# one hot encoding
venues_toronto_hc = pd.get_dummies(data = venues_toronto, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
venues_toronto_hc.head()
#venues_toronto['Venue Category'].uniques()

Load the dataframe for further analysis

In [ ]:
features = ['Neighbourhood',
            'Zoo Exhibit','Zoo','Park','Dessert Shop','Burger Joint','Campground','Hakka Restaurant','Coffee Shop',
            'Fried Chicken Joint','Sandwich Place','Italian Restaurant','Chinese Restaurant','BBQ Joint','Steakhouse',
            'Golf Course','Liquor Store','Sports Bar','Food & Drink Shop','Pub','Beach','Pharmacy','Sporting Goods Shop',
            'Supermarket','Arts & Crafts Store','Grocery Store','Toy / Game Store','Breakfast Spot','Noodle House',
            'Community Center','Ice Cream Shop','Fast Food Restaurant','Bookstore','Gym','American Restaurant','Tea Room',
            'Cosmetics Shop','Wings Joint','Sushi Restaurant','Seafood Restaurant','Gym / Fitness Center','Fish & Chips Shop',
            'Restaurant','Pizza Place','Bistro','Smoothie Shop','African Restaurant','Thai Restaurant','Hotpot Restaurant',
            'Café','History Museum','Clothing Store','Gastropub','Indie Movie Theater','Discount Store','Shopping Mall',
            'Bagel Shop','Other Great Outdoors','Movie Theater','Playground','General Entertainment','Gourmet Shop',
            'Comic Shop','Farmers Market','Beer Bar','Burrito Place','Cocktail Bar','Historic Site','French Restaurant',
            'Trail','Taco Place','Diner','Skating Rink','Bubble Tea Shop','Auto Dealership','Shopping Plaza','Food','Road',
            'New American Restaurant','Rock Climbing Spot','Hobby Shop','Furniture / Home Store',
            'Paper / Office Supplies Store','Deli / Bodega','Health Food Store','Ski Chalet','Hardware Store',
            'Recreation Center','Garden','Electronics Store','Department Store','Juice Bar','Museum','Food Court','Bar','Theater',
            'Snack Place','Soup Place','Train Station','Theme Park','Theme Park Ride / Attraction','Shoe Store',
            'Chocolate Shop','Tapas Restaurant','Organic Grocery','Whisky Bar','Comedy Club','Nightclub',
            'Convenience Store']

venues = venues_toronto_hc[features].groupby('Neighbourhood').sum()

venues.head()

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 7, random_state = 0).fit(venues)

In [ ]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = venues.columns
means_df.index = ['Group 1','Group 2','Group 3','Group 4', 'Group 5', 'Group 6', 'Group 7']
means_df['Total Sum'] = means_df.sum(axis = 1)
mean = means_df['Total Sum']
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

In [ ]:
neigh_summary = pd.DataFrame([venues.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighbourhood', 'Group']

In [ ]:
a = neigh_summary[neigh_summary['Group'] == 5]
a = neigh_summary[neigh_summary['Group'] == 2]

In [ ]:
def locate_best(neighbourhood,group):
    ad = []
    counter = 0
    for neighbourhood, group in zip(neighbourhood,group):
         
        # make the GET request
        ad_dict = {}
        ad_dict['Neighbourhood'] = neighbourhood; 
        ad_dict['Group'] = group
        ad.append(ad_dict)
        counter += 1
    return ad;

In [ ]:
best = locate_best(neigh_summary[neigh_summary['Group'] == 5],neigh_summary[neigh_summary['Group'] == 5])
print(list(best))

In [ ]:
name_of_neigh = neigh_summary[neigh_summary['Group'] == 5]
a.head()
#venues[venues['Neighbourhood'] == name_of_neigh].iloc[0,1:5].to_dict()

In [ ]:
prefs_df = pd.merge(venues_toronto, name_of_neigh, on='Neighbourhood')

In [ ]:
prefs_df.reset_index(drop=True).drop(columns = 'Unnamed: 0').head()

In [ ]:
from folium import IFrame
#lat = 43.72021610 ; lon = -79.5395769
map_Toronto_Central = folium.Map(location=[lat, lon], zoom_start=10.5)

# Add a popup label for the map
text = 'Proximal neighbourhoods to stay in Central Toronto'
iframe = folium.IFrame(text, width=700, height=50)
popup = folium.Popup(iframe, max_width=1000)
Text = folium.Marker(location=[lat,lon], popup=popup,
                     icon=folium.Icon(icon_color='green'))

# add markers to map
for lat, lng, label in zip(prefs_df['Latitude'], prefs_df['Longitude'], prefs_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng], radius = 6, popup = label, color ='red', fill = True,fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(Toronto_boroughs_map)

Toronto_boroughs_map.add_child(Text)
Toronto_boroughs_map.save('Destination Toronto_Central.html')
Toronto_boroughs_map
